# 뉴스 기사의 댓글 모으기 

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os

In [4]:
query_txt = '뉴스기사댓글'
query_url = input('뉴스 URL 입력: ')

뉴스 URL 입력: https://n.news.naver.com/mnews/ranking/article/015/0004749628?ntype=RANKING&sid=001


In [5]:
cnt = int(input('크롤링 할 건수(10건 단위로 입력): '))
page_cnt = math.ceil(cnt / 20)    # math.ceil : 소수값이 존재할 시 올림

f_dir = input("파일을 저장할 폴더명(예:c:\\temp\\):")

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xlsx'

# C:\Users\socsoft\Downloads\internship_kwj\

크롤링 할 건수(10건 단위로 입력): 30
파일을 저장할 폴더명(예:c:\temp\):C:\Users\socsoft\Downloads\internship_kwj\


In [6]:
s_time = time.time()

path = 'C:/Users/SOCSOFT/Downloads/internship_kwj/chromedriver.exe'
driver = webdriver.Chrome(path)

driver.get(query_url)
time.sleep(5)

In [7]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result = soup.find('div', class_='u_cbox_head').find('span', 'u_cbox_count')
#result = soup.find('ul', 'u_cbox_list')
result2 = result.get_text()
result3 = re.search('\d+', result2)
search_cnt = int(result3.group())

# 총 댓글 수를 확인하고 요청 건수화 비교 후 동기화
if cnt > search_cnt:
    cnt = search_cnt
    
print("전체 검색 결과 건수 :",search_cnt,"건")
print("실제 최종 출력 건수",cnt)
print("실체 출력될 최종 페이지수" , page_cnt)

# 댓글 중 일부 내용만 보일 경우 전체 내용 수집

# 최초 10건 수집후 댓글 더보기 클릭
# 아래 버튼 클릭, 첫 화면에 20건의 댓글 출력
driver.find_element_by_xpath('''//*[@id="cbox_module"]/div[2]/div[9]/a/span[1]''').click()
time.sleep(3)

전체 검색 결과 건수 : 155 건
실제 최종 출력 건수 30
실체 출력될 최종 페이지수 2


In [8]:
# 댓글 관련 내용 수집

now1 = time.localtime()
s1 = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

writer_id_li =[]
review_li = []
write_date_li = []
recomm_li = []
unrecomm_li = []

if cnt <= 20:
    f = open(ff_name, 'a', encoding='UTF-8')    # append
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    count = 0
    
    comm_result = soup.find('div', 'u_cbox_content_wrap').find('ul')
    comm_all = comm_result.find_all('li')
    
    for li in comm_all:
        count += 1
        print("\n")
        print("%s 번째 댓글 수집 중..." % count)
        
        writer_id = li.find('span', 'u_cbox_nick').get_text()
        print('1.작성자ID:' + writer_id)
        f.write("1.작성자ID:" + writer_id + "\n")
        writer_id_li.append(writer_id)
        
        try:
            review = li.find('span', 'u_cbox_contents').get_text()
        except AttributeError:
            review = '작성자에 의해 삭제된 댓글입니다.'
            print('2.리뷰:', review)
        else:
            print('2.리뷰:', review)
        f.write('2.리뷰:' + review + "\n")
        review_li.append(review)
        
        write_date = li.find('span', 'u_cbox_date').get_text()
        print('3.작성일자:',write_date)
        f.write("3.작성일자:" + write_date + "\n")
        write_date_li.append(write_date)
            
        try:
            recomm = li.find('div', 'u_cbox_recomm_set').find_all('em')    
        
            recomm1 = recomm[0].text
            print('4.공감:', recomm1)
        except AttributeError:
            recomm1 = '0'
            print('4.공감:', recomm1)
        except IndexError:
            recomm1 = '0'
            print('4.공감:', recomm1)
        f.write('4.공감:'+ recomm1 + '\n')
        recomm_li.append(recomm1)

        try:
            unrecomm = recomm[1].text
            print('5.비공감:', unrecomm)
        except IndexError:
            unrecomm = '0'
            print('5.비공감:', unrecomm)
        f.write('5.비공감:' + unrecomm + '\n')
        unrecomm_li.append(unrecomm)

        time.sleep(0.2)

        if count == cnt:
            break
            
    time.sleep(random.randrange(3,8))
    
# 위 코드까지는 20건까지만 완료
# 20건 초과 시 아래 작업

else:
    i = 1
    
    while (i <= page_cnt-1):
        driver.find_element_by_xpath('''//*[@id="cbox_module"]/div[2]/div[9]/a''').click()
        time.sleep(3)
        i += 1
        
    f = open(ff_name, 'a', encoding='UTF-8')
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    count = 0
    
    comm_result = soup.find('div', 'u_cbox_content_wrap').find('ul')
    comm_all = comm_result.find_all('li')
    
    for li in comm_all:
        count += 1
        print("\n")
        print("%s 번째 댓글 수집 중..." % count)
        
        writer_id = li.find('span', 'u_cbox_nick').get_text()
        print('1.작성자ID:' + writer_id)
        f.write("1.작성자ID:" + writer_id + "\n")
        writer_id_li.append(writer_id)
        
        try:
            review = li.find('span', 'u_cbox_contents').get_text()
        except AttributeError:
            review = '작성자에 의해 삭제된 댓글입니다.'
            print('2.리뷰:', review)
        else:
            print('2.리뷰:', review)
        f.write('2.리뷰:' + review + "\n")
        review_li.append(review)
        
        write_date = li.find('span', 'u_cbox_date').get_text()
        print('3.작성일자:',write_date)
        f.write("3.작성일자:" + write_date + "\n")
        write_date_li.append(write_date)
            
        try:
            recomm = li.find('div', 'u_cbox_recomm_set').find_all('em')    
        
            recomm1 = recomm[0].text
            print('4.공감:', recomm1)
        except AttributeError:
            recomm1 = '0'
            print('4.공감:', recomm1)
        except IndexError:
            recomm1 = '0'
            print('4.공감:', recomm1)
        f.write('4.공감:'+ recomm1 + '\n')
        recomm_li.append(recomm1)

        try:
            unrecomm = recomm[1].text
            print('5.비공감:', unrecomm)
        except IndexError:
            unrecomm = '0'
            print('5.비공감:', unrecomm)
        f.write('5.비공감:' + unrecomm + '\n')
        unrecomm_li.append(unrecomm)

        time.sleep(0.2)

        if count == cnt:
            break
            
    time.sleep(random.randrange(3,8))



1 번째 댓글 수집 중...
1.작성자ID:the_****
2.리뷰: 벤츠 차주 트럭인듯
3.작성일자: 2022.09.15. 08:39
4.공감: 453
5.비공감: 0


2 번째 댓글 수집 중...
1.작성자ID:hide****
2.리뷰: 나라는 점점 선진화가 되어가는데 왜 저런것들 뇌구조는 점점 쇠퇴해가는지 모르겠다.
3.작성일자: 2022.09.15. 08:38
4.공감: 146
5.비공감: 1


3 번째 댓글 수집 중...
1.작성자ID:jiny****
2.리뷰: 같은 사람 아니야?
3.작성일자: 2022.09.15. 08:40
4.공감: 79
5.비공감: 0


4 번째 댓글 수집 중...
1.작성자ID:namh****
2.리뷰: 상식없는 나라가 되어버렸네
3.작성일자: 2022.09.15. 08:38
4.공감: 31
5.비공감: 0


5 번째 댓글 수집 중...
1.작성자ID:sexy****
2.리뷰: 탑차는 길어서 저리 주차하는건가????
3.작성일자: 2022.09.15. 08:38
4.공감: 18
5.비공감: 1


6 번째 댓글 수집 중...
1.작성자ID:namg****
2.리뷰: 2.5톤 이상 화물차는 아파트 못되는데!!!!!딱바도 차고지에다가 대야하는데.
꼭 신고해서.
과태료 물게 하라!!!!!!!
3.작성일자: 2022.09.15. 09:20
4.공감: 14
5.비공감: 1


7 번째 댓글 수집 중...
1.작성자ID:byun****
2.리뷰: 트럭은 차고지가 따로잇습니다. 트럭은 경찰고발하면  과태료입니다.
3.작성일자: 2022.09.15. 09:26
4.공감: 10
5.비공감: 0


8 번째 댓글 수집 중...
1.작성자ID:zzon****
2.리뷰: 저거 벤츠차주가 꼴받아서 복수하는거같은데ㅋ
3.작성일자: 2022.09.15. 08:48
4.공감: 9
5.비공감: 0


9 번째 댓글 수집 중...
1.작성자ID:shar****
2.리뷰: 적당히 기자노릇이라도 해라.. 차주도좀  알아보고 이유도 물어보고 경찰도 만나

In [10]:
# xls 와 csv 로 저장하기

news_comm = pd.DataFrame()
news_comm['작성자ID']=pd.Series(write_date_li)
news_comm['리뷰내용']=pd.Series(review_li)
news_comm['작성일자']=pd.Series(write_date_li)
news_comm['공감횟수']=pd.Series(recomm_li)
news_comm['비공감횟수']=pd.Series(unrecomm_li)

# csv 형태로 저장하기
news_comm.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
news_comm.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

print("\n") 
print("1.요청된 총 %s 건의 리뷰 중 실제 크롤링 된 리뷰수는 %s 건" %(cnt,count))
print("2.총 소요시간은 %s 초 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xlsx 파일명 : %s " %fx_name)
print("=" *80)


driver.close( )



1.요청된 총 30 건의 리뷰 중 실제 크롤링 된 리뷰수는 30 건
2.총 소요시간은 48.5 초 
3.파일 저장 완료: txt 파일명 : C:\Users\socsoft\Downloads\internship_kwj\2022-09-15-10-15-21-뉴스기사댓글\2022-09-15-10-15-21-뉴스기사댓글.txt 
4.파일 저장 완료: csv 파일명 : C:\Users\socsoft\Downloads\internship_kwj\2022-09-15-10-15-21-뉴스기사댓글\2022-09-15-10-15-21-뉴스기사댓글.csv 
5.파일 저장 완료: xlsx 파일명 : C:\Users\socsoft\Downloads\internship_kwj\2022-09-15-10-15-21-뉴스기사댓글\2022-09-15-10-15-21-뉴스기사댓글.xlsx 
